<a href="https://colab.research.google.com/github/MarzoliLeo/UE-Text-To-3D/blob/main/Text_To_3D_Model_Methaporalab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting di Google Drive e verifica della presenza di Point-E

In [8]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import os

# Percorso al repository clonato
repo_path = '/content/drive/MyDrive/point-e-main'
os.chdir(repo_path)


In [10]:
!ls /content/drive/MyDrive/


'Attività di pratica - Fare Innovazione con l'\''AI | Tools 0-10x.gdoc'
'Colab Notebooks'
 DocumentoTecnico-SmartGlasses.gdoc
 DocumentoTecnico-SmartGlasses.pdf
 point-e-main


In [11]:
!pip install -e .


Obtaining file:///content/drive/MyDrive/point-e-main
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-9cj32hz5/clip_0987eff19eab4ab9b0dd7f0646e8c93b
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-9cj32hz5/clip_0987eff19eab4ab9b0dd7f0646e8c93b
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.2 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=29043b46fca7fa3a5bfc4ce5b87aa12431729bc54a29feed12ea0778fa07aa61
  Stored in directory: /tmp/pip-ephem-wheel-cache-pqs3424u/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
  Created wheel for fire: filen

# Programmi



1° prova, va comunque eseguita per alcuni import che si usano dopo.

In [12]:
# STEP 2: Text Encoder (CLIP) + Prompt Embedding
import torch
import clip
from PIL import Image
import numpy as np

# Scegli il device (usa GPU se disponibile)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Carica il modello CLIP pre-addestrato
model, preprocess = clip.load("ViT-B/32", device=device)

# Prompt testuale di esempio
prompt = "a red futuristic sci-fi chair made of metal"

# Codifica il prompt in un vettore latente
with torch.no_grad():
    text_tokens = clip.tokenize([prompt]).to(device)
    text_features = model.encode_text(text_tokens)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)  # Normalizza

# Stampa dimensioni del vettore (512)
print(f"Text embedding shape: {text_features.shape}")


100%|███████████████████████████████████████| 338M/338M [00:07<00:00, 45.4MiB/s]


Text embedding shape: torch.Size([1, 512])


2° Prova


# Utilizzo di OpenAI Point-E per creare da un testo una point cloud.

In [15]:
from tqdm.auto import tqdm
import open3d as o3d

from point_e.diffusion.configs import DIFFUSION_CONFIGS, diffusion_from_config
from point_e.diffusion.sampler import PointCloudSampler
from point_e.models.configs import MODEL_CONFIGS, model_from_config
from point_e.models.download import load_checkpoint
from point_e.util.plotting import plot_point_cloud

# 🔧 Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ✅ Load base model
print('Caricamento modello base...')
base_name = 'base40M-textvec'
base_model = model_from_config(MODEL_CONFIGS[base_name], device)
base_model.load_state_dict(load_checkpoint(base_name, device))
base_model.eval()
base_diffusion = diffusion_from_config(DIFFUSION_CONFIGS[base_name])

# ✅ Load upsampler model
print('Caricamento modello di upsampling...')
upsampler_model = model_from_config(MODEL_CONFIGS['upsample'], device)
upsampler_model.load_state_dict(load_checkpoint('upsample', device))
upsampler_model.eval()
upsampler_diffusion = diffusion_from_config(DIFFUSION_CONFIGS['upsample'])

# ✅ Sampler
sampler = PointCloudSampler(
    device=device,
    models=[base_model, upsampler_model],
    diffusions=[base_diffusion, upsampler_diffusion],
    num_points=[1024, 4096 - 1024],
    aux_channels=['R', 'G', 'B'],
    guidance_scale=[3.0, 0.0],
    model_kwargs_key_filter=('texts', ''),
)


Caricamento modello base...
Caricamento modello di upsampling...


## Sezione dove viene definito il prompt

In [16]:
# ✏️ Prompt testuale
prompt = 'a red futuristic sci-fi chair made of metal'

# 🔁 Generazione progressiva
samples = None
for x in tqdm(sampler.sample_batch_progressive(batch_size=1, model_kwargs=dict(texts=[prompt]))):
    samples = x

0it [00:00, ?it/s]

## Estrazione della point cloud e suo salvataggio in Drive.

In [17]:
# ✅ Estrai la point cloud
pc = sampler.output_to_point_clouds(samples)[0]

# 💾 Salva come PLY in Google Drive
output_dir = '/content/drive/MyDrive/point_e_output'
os.makedirs(output_dir, exist_ok=True)
ply_path = os.path.join(output_dir, 'generated_object.ply')

In [18]:
# 🔄 Conversione e salvataggio con Open3D (corretto)
points = np.array(pc.coords)

# Estrai R, G, B dai canali
R = np.array(pc.channels['R']) * 255
G = np.array(pc.channels['G']) * 255
B = np.array(pc.channels['B']) * 255

colors = np.stack([R, G, B], axis=1).astype(np.uint8)

# Crea point cloud Open3D
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors / 255.0)

# Salva come PLY
ply_path = os.path.join(output_dir, 'generated_object.ply')
o3d.io.write_point_cloud(ply_path, pcd)
print(f"✅ Point cloud salvata in formato PLY in: {ply_path}")

# (Opzionale) Salva come OBJ — ma OBJ non supporta colori nei punti, solo mesh o .mtl
#bj_path = os.path.join(output_dir, 'generated_object.obj')
#o3d.io.write_point_cloud(obj_path, pcd, write_ascii=True)
#print(f"✅ (Opzionale) Point cloud salvata in formato OBJ in: {obj_path}")



✅ Point cloud salvata in formato PLY in: /content/drive/MyDrive/point_e_output/generated_object.ply
[Open3D WARNING] Write geometry::PointCloud failed: unknown file extension obj for file /content/drive/MyDrive/point_e_output/generated_object.obj.
✅ (Opzionale) Point cloud salvata in formato OBJ in: /content/drive/MyDrive/point_e_output/generated_object.obj


## Fase di merge della point Cloude per ottenere una mesh.

In [20]:
import trimesh
from point_e.util.pc_to_mesh import marching_cubes_mesh

# 📂 Directory di output
output_dir = "/content/drive/MyDrive/3d_output"
os.makedirs(output_dir, exist_ok=True)

In [21]:
# === APPROCCIO A: MARCHING CUBES + SDF MODEL (Point-E)
# 🧠 Più qualità grazie all'uso del modello SDF
USE_MARCHING_CUBES_SDF = True
if USE_MARCHING_CUBES_SDF:
    print("🔷 Generazione Mesh con Marching Cubes + SDF model...")

    # Carica modello SDF da Point-E
    from point_e.models.configs import MODEL_CONFIGS, model_from_config
    from point_e.models.download import load_checkpoint

    print('📥 Caricamento modello SDF...')
    sdf_model_name = 'sdf'
    sdf_model = model_from_config(MODEL_CONFIGS[sdf_model_name], device)
    sdf_model.load_state_dict(load_checkpoint(sdf_model_name, device))
    sdf_model.eval()

    # Crea la mesh usando il modello SDF (migliore qualità)
    from point_e.util.pc_to_mesh import marching_cubes_mesh
    mesh = marching_cubes_mesh(
        pc=pc,
        model=sdf_model,
        batch_size=4096,
        grid_size=128,  # puoi usare 32 per velocità o 128 per alta qualità
        progress=True,
    )

    # Salva in formato .ply (puoi anche esportare in .obj se necessario)
    mesh_path = os.path.join(output_dir, "mesh_marching_sdf.ply")
    with open(mesh_path, 'wb') as f:
        mesh.write_ply(f)
    print(f"✅ Mesh salvata (marching cubes + SDF): {mesh_path}")

🔷 Generazione Mesh con Marching Cubes + SDF model...
📥 Caricamento modello SDF...


  0%|          | 0.00/38.0M [00:00<?, ?iB/s]

  0%|          | 0/512 [00:00<?, ?it/s]

✅ Mesh salvata (marching cubes + SDF): /content/drive/MyDrive/3d_output/mesh_marching_sdf.ply


In [22]:
# === APPROCCIO B: POISSON RECONSTRUCTION (Open3D)
# 🌊 Superficie più "morbida", ma richiede point cloud densa
USE_POISSON_RECONSTRUCTION = True
if USE_POISSON_RECONSTRUCTION:
    print("🔷 Generazione Mesh con Poisson Reconstruction (Open3D)...")

    # Ricrea point cloud Open3D (se non esiste)
    points = np.array(pc.coords)
    R = np.array(pc.channels['R']) * 255
    G = np.array(pc.channels['G']) * 255
    B = np.array(pc.channels['B']) * 255
    colors = np.stack([R, G, B], axis=1).astype(np.uint8)

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors / 255.0)

    # 🔍 Stima delle normali (necessaria per Poisson)
    pcd.estimate_normals()
    pcd.orient_normals_consistent_tangent_plane(10)

    # 🌀 Ricostruzione mesh
    mesh_p, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=8
    )

    # ✂️ Pulizia: rimuove mesh isolate lontane dal centro
    bbox = pcd.get_axis_aligned_bounding_box()
    mesh_crop = mesh_p.crop(bbox)

    # 💾 Salva mesh in formato .ply per coerenza
    ply_path_poisson = os.path.join(output_dir, "mesh_poisson.ply")
    o3d.io.write_triangle_mesh(ply_path_poisson, mesh_crop)
    print(f"✅ Mesh salvata (poisson, .ply): {ply_path_poisson}")



🔷 Generazione Mesh con Poisson Reconstruction (Open3D)...
[Open3D WARNING] Write Ply clamped color value to valid range
✅ Mesh salvata (poisson, .ply): /content/drive/MyDrive/3d_output/mesh_poisson.ply
